In [3]:
from pose_graph import PoseGraph
import numpy as np

In [5]:
sensor_id = "sensor_1"
print( "The sensor "+ sensor_id + " is already in the graph \n please use update_graph() to update the edges")

The sensor sensor_1 is already in the graph 
 please use update_graph() to update the edges


In [1]:
sensor_id = ["cam0", "cam1", "imu0", "imu1"]

pose_id = ["cam1", "cam2", "cam3", "cam4", "cam5"]

poses = {
    "cam1": [(1, "M"), (2, "N"), (3, "O"), (4, "P"), (5, "Q")],
    "cam2": [(2, "N")],
    "cam3": [(3, "O")],
    "cam4": [(4, "P")],
    "cam5": [(5, "Q")],
}

# for pose in poses.items():
#     print(pose[1][1])

if 1 not in poses["cam1"]:
    print("not in")


SyntaxError: invalid syntax (30715001.py, line 7)